# CCS 2017 - Informatique pour tous
-------------------------------------------------

In [ ]:
import math
import numpy as np
import random

## I Création d'une exploration et gestion des points d'intérêts  

### *I.A - Génération d'une exploration d'essai*

**I.A.1)** a) Il y a plusieurs manières de traiter cette question. On peut tout d'abord penser à une boucle `for` après avoir créé un tableau de taille (n,2). Mais les points devant être deux à deux distincts, on va plutôt utiliser une boucle `while` pour tester la taille du tableau que l'on augmentera à chaque étape. Pour vérifier qu'un point appartient ou non au tableau, nous utiliserons la commande `in` rappelée dans l'annexe. Une fois cette démarche générale fixée, il faut choisir entre manipuler des listes pyhton (et la méthode `.append`) ou des tableaux numpy (et la fonction `concatenate` du module `numpy`.) On donne ici les deux versions, mais l'idée générale du sujet est plus de se familiariser avec les tableaux numpy.

Le tableau final PI est une matrice, i.e. un tableau à *deux dimensions* (contrairement à un vecteur qui est un tableau à une dimension). Pour concaténer des tableaux, ils doivent avoir la même dimension. Le nouveau point que l'on crée doit donc être de dimension $2$ i.e. une *matrice* a une ligne et deux colonnes. De même, le tableau vide que l'on crée initialement doit être de dimension $2$ et avoir le bon nombre de lignes et de colonnes (la bonne forme, *shape* en anglais). Dans notre cas, $0$ ligne (initialisation, pas encore de données), et $2$ colonnes (dimension des futurs données).


In [ ]:
# Version tableaux numpy

def générer_PI(n:int, cmax:int) -> np.ndarray:

    PI = np.empty((0,2)) # PI est de dimension 2 
    
    while len(PI)<n:
        x = random.randrange(0,cmax+1)
        y = random.randrange(0,cmax+1)
        nouveau_point = np.array([[x,y]]) # matrice à 1 ligne 2 colonnes
        
        if not(nouveau_point in PI):
            PI = np.concatenate((PI,nouveau_point))
    
    return PI

La version avec les listes est plus simple car il y a moins à se soucier de ces comparibilité de dimension lors de l'ajout. A la fin, on transfome la liste en un tableau numpy à l'aide du constructeur `np.array`.

In [ ]:
# Version utilisant des listes

def générer_PI(n:int, cmax:int) -> np.ndarray:
    
    PI = []
    
    while len(PI)<n:
        x = random.randrange(0,cmax+1)
        y = random.randrange(0,cmax+1)
        nouveau_point = [x,y] 
        
        if not(nouveau_point in PI):
            PI.append(nouveau_point)
    
    return np.array(PI) 

**I.A.1)** b) Les arguments de la fonction `générer_PI` doivent être *positif ou nul*. De plus, il y a $cmax + 1$ entiers distincts inférieurs ou égaux à $cmax$. Il existe donc $(cmax+1)^2$ points d'intérets distincts. Il faut donc que $n\leqslant (cmax+1)^2$.

**I.A.2)** Pour ne pas avoir à distinguer de cas, il est judicieux d'ajouter la position du robot aux points d'intérêts, en position $n$. On remarquera aussi que la matrice de distances est *symétrique* et possède une *diagonale de zéros*, ce qui évite des calculs et explique l'initialisation à l'aide de la commande `np.zeros`.

Il est pertinant ici de créer une fonction auxiliaire `distance` d'entête :

    def distance(A:np.ndarray,B:np.ndarray) -> float:
    
prenant en argument deux vecteurs A et B du plan et renvoyant leur distance euclidienne. 

> C'est un grand classique des sujets d'informatique


In [ ]:
# Fonctions auxiliaires

def distance(A:np.ndarray,B:np.ndarray) -> float:
    return math.sqrt((B[0]-A[0])**2 + (B[1]-A[1])**2)

# Fonction principale

def calculer_distances(PI:np.ndarray) -> np.ndarray:
    
    n = len(PI)
    D = np.zeros((n+1,n+1)) # matrice des distances
    
    x,y = position_robot() 
    position_du_robot = np.array([[x,y]]) # matrice à 1 ligne et 2 colonnes
    PI = np.concatenate((PI,position_du_robot)) 
    
    for i in range(n+1):
        for j in range(i+1): # il y a des 0 sur la diagonale de D
            d = distance(PI[i],PI[j])
            D[i,j] = d
            D[j,i] = d # la matrice des distance est symétrique
    
    return D 

Pour calculer la distance entre A et B, on peut également utiliser l'aspect *universel* des tableaux numpy qui permet des codes très concis, convenant pour toute taille de A et B:

In [ ]:
def distance(A:np.ndarray,B:np.ndarray) -> float:
    return np.sqrt(sum(A-B)**2) 

>**Attention !** le même code avec des listes ne fonctionne *pas du tout* car L1 - L2 renvoie une erreur et L1 + L2 concatène les listes...

## *I.B - Traitement d'image*

**I.B.1)** La fonction `F1` détecte la plus petite et la plus grande des intensités des pixels de l'image (lignes $2$ et $3$), puis compte le nombre de pixels possèdant une intensité comprise entre ces deux extrèmes. Ceci est fait dans la boucle `for` via l'itérateur `photo.flat` qui parcourt les valeurs des pixels de la photo, en augmentant de $1$ l'élément d'indice $n-p$ d'une liste `h` lorsque l'intensité $p$ est rencontrée. A la fin, la liste `h` contient en position $n-p$ le nombre de pixels de l'image d'intensité $p$. 

**I.B.2)** On va parcourt les points définissant la photo à l'aide de deux boucles `for`. Si le point courant vérifies les conditions, ce point est ajouté aux points d'intérêts via la commande `concatenate`, à partir d'un tableau vide créé via la fonction `np.empty`. 

In [ ]:
# Version tableau numpy

def sélectionner_PI(photo:np.ndarray, imin:int, imax:int) -> np.ndarray:
    
    PI = np.empty((0,2))
    m,n = photo.shape()
    
    for i in range(m):
        for j in range(n):
            if imin <= photo[i,j] <= imax:
                PI = np.concatenate((PI,[[i,j]]))
                
    return PI

Ici encore, si l'on est plus à l'aise avec les listes, on peut travailler avec celles-ci, puis créer in fine un tableau via la fonction `np.array`.

In [ ]:
# Version avec des listes

def sélectionner_PI(photo:np.ndarray, imin:int, imax:int) -> np.ndarray:
    
    L = []
    m,n = photo.shape()
    
    for i in range(m):
        for j in range(n):
            if imin <= photo[i,j] <= imax:
                L = L.append([i,j])
                
    return np.array(L)

## *I.C - Base de données*

**I.C.1)** Une exploration est en cours si elle a commencée, et si elle n'est pas terminée. 

    SELECT EX_NUM FROM EXPLO
    WHERE EX_DEB IS NOT NULL AND EX_FIN IS NULL

*On utilise le signe = pour comparer des valeurs numériques. Il faut utiliser ici `IS NULL` ou `IS NOT NULL`.*

**I.C.2)** Les ... sont a remplacer par le numéro connu de l'exploration 

    SELECT PI_NUM,PI_X,PI_Y FROM PI
    WHERE EX_NUM = ...


**I.C.3)** Pour une exploration donnée, la plus petit rectangle contenant les points explorés a pour surface $(\max(PI_{\_}X)-\min(PI_{\_}X))*(\max(PI_{\_}Y)-\min(PI_{\_}Y))$. Il faut ensuite convertir en $m^2$ car les données initiales sont en milimètre.  

On va utiliser des *fonctions d'agrégations* (`MIN` et `MAX`), donc la commande `GROUP BY`. Les informations sont dans plusieurs tables, il faudra donc utiliser une jointure via la commande `JOIN` :

    SELECT EX_NUM, (MAX(PI_X)-MIN(PI_X))*(MAX(PI_Y)-MIN(PI_Y))*0.000001 AS SURFACE
    FROM EXPLO JOIN PI ON EXPLO.EX_NUM = PI.EX_NUM
    WHERE EX_FIN IS NOT NULL
    GROUP BY EX_NUM

**I.C.4)** La réponse à cette question dépend de la *représentation binaire* choisie pour les entiers. Si ceux-ci sont représentés par des entiers signés sur $32$ bits (ce qui est le plus courant), le plus grand entier stockable  est donc égal à $2^{31}-1 = 2 147 483 647$. La surface maximale d'une zone d'exploration est donc égale à $(2^{31}-1)^2$ soit environ 4.6 millions de $km^2$.  (Pour information, la superficie de la France est d'environ 643 000 $km^2$). 

**I.C.5)** On va utiliser des *fonctions d'agrégation* (`COUNT` et `SUM`) donc un `GROUP BY` (par instrument). Il faut identifier les différentes tables contenant les informations pertinentes (`EXPLO`, `ANALY`, `INTYP` et `INSTR`) et les joindre. Enfin il faut se limiter à l'exploration en cours via la commande `WHERE`. 

    SELECT COUNT(IN_NUM) AS NB_UTILISATIONS, SUM(IT_DUR) AS DUREE_UTILISATIONS
    FROM EXPLO
        JOIN ANALY 
            ON EXPLO.EX_NUM = ANALY.EX_NUM
                JOIN INTYP 
                    ON ANALY.TY_NUM = INTYP.TY_NUM
                        JOIN INSTR 
                            ON INTYP.IN_NUM = INSTR.IN_NUM
    WHERE EX_DEB IS NOT NULL AND EX_FIN IS NULL
    GROUP BY IN_NUM

# II Planification d'une exploration : première approche 

## *II.A - Quelques fonctions utilitaires*

**II.A.1)** On va sommer, à l'aide d'une boucle `for`, les différentes distances en mettant à jour l'indice de la position du robot.

In [ ]:
def longueur_chemin(chemin:list, d:np.array) -> float:
    
    # Position initiale du robot en dernière ligne de la matrice
    position = len(d)-1  
    
    l = 0
    for i in chemin:
        l += d[position,i]
        position = i
    
    return l

**II.A.2)** La fonction est divisée en deux étapes : la suppression des doublons puis l'ajouts des points manquants. On va de nouveau utiliser la fonction `in` pour les listes. 

In [ ]:
def normaliser_chemin(chemin:list, n:int) -> list:
    
    # Suppression des doublons 
    
    valide = []
    
    for i in chemin:
        if i not in valide and i< n:
            valide.append(i)
            
    # Ajout des chemins manquants par ordre croissant
    
    for k in range(n):
        if k not in valide:
            valide.append(k)
            
    return valide  

## *II.B - Force brute*

**II.B.1)** Pour le premier point, on a $n$ choix, puis $n-1$ choix pour le deuxième, etc... Le nombre de chemins possible est donc égal à $n!$.  
**II.B.2)** *C'est une question classique à relier au capacités de l'ordinateur (mémoire, complexité temporelle,...)* On a $n!\simeq 2.43\times10^{18}$. A une fréquence de  3GHz, il faudrait donc au moins $2.43\times10^{18}/3\times10^9$ secondes pour traiter ces données i.e. environ $25$ ans, ce qui dépace largement la durée de vie du robot. 
**II.C.1)**

## *II.C - Algorithme du plus proche voisin*

**II.C.1)** On a utiliser une fonction auxiliaire dont l'entête est :

    def PI_suivant(i:int, d:np.array, PI_explorés:list) -> int
   
prennant comme argument l'indice du point d'intérêt que l'on vient d'explorer, la matrice des distances $d$ et la liste des points d'intérêts déjà explorés, et qui renvoit l'indice du plus proche point d'intérêt à explorer.

In [ ]:
# Fonction auxilliaire

def PI_suivant(i:int, d:np.array, PI_explorés:list) -> int:
    ''' Renvoie l'indice du point d'intérêt suivant à explorer'''
    
    # Un point d'intérêt de moins que la taille de la matrice des distances
    n = len(d)-1

    # On élimine les points déjà explorés
    PI_restants = [j for j in range(n) if j not in PI_explorés]
    
    d_min = np.inf # pour que la première comparaison de a boucle soit vraie
    
    for j in PI_restants:
        if d[i,j]<d_min:
            indice = j
            d_min = d[i,j]
    
    return indice

            
# Fonction principale

def plus_proche_voisin(d:np.array) -> list:
    
    # Un point d'intérêt de moins que la taille de la matrice des distances
    n = len(d)-1
    # Position du robot en dernière ligne de la matrice 
    position = n
    
    chemin = []

    for k in range(n):
        position = PI_suivant(position, d, chemin)
        chemin.append(position)
    
    return chemin

*Noter l'utilisation de `np.inf` pour l'initialisation de `d_min` ligne 14 et l'utilisation d'une **condition dans une compréhension de liste** via le mot `if` ligne 11.*

**II.C.2)** Notons $n$ le nombres de points d'intérêts à explorer. Il faut détailler les complexités fonctions par fonctions :   
* fonction `calculer_distances` :
    - l'initialisation de la matrice (écriture des zéros) est en $O(n^2)$ car on écrit $(n+1)^2$ zéros.
    - les deux boucles `for` imbriquées, parcourues au plus $n+1$ fois chacune en $O(1)$ donne une complexité en $O(n^2)\times O(1) = O(n^2)$.  
    
La complexité de la fonction `calculer_distance` est donc en $O(n^2)$.
* fonction `plus_proche_voisin` :
    - $n$ appels à la fonction `PI_suivant`.
    - la fonction `PI_suivant` possède une compréhension de liste en $O(n^2)$ car la commande `in` a une complexité linéaire, et une boucle `for` parcourue en $O(n)$.  
    
La complexité de la fonction `plus_proche_voisin` est donc de $n*(O(n) + O(n)) = O(n^2)$.  
 
On fait un appel à la fonction `calculer_distance` et $n$ appels à la fonction `plus_proche_voisin`. La complexité globale est donc en $O(n^2) + n\times O(n^2) = O(n^3)$.

**I.C.3)**  Prenons comme point de départ le point $(0,4000)$. L'algorithme du plus proche voisin fournit alors comme chemin $[1,0,2]$ (de distance 11 mètres) alors que le chemin $[2,1,0]$ ne fait que $10$ mètres. 

# III Deuxième approche : algorithme génétique

## *III.A - Initialisation et évaluation*

**Attention**, il y a une erreur ici : la longueur n'est pas un entier mais un flottant. On va créer une fonction auxiliaire  

    créer_individus(d:np.array) -> tuple

prenant en argument le tableau des distances et renvoiant un individu au hasard grace à la fonction `random.shuffle`, puis appeler $m$ fois cette fonction. 

In [ ]:
# Fonction auxilliaire

def créer_individu(d:np.array) -> (float,list):
    '''Renvoie un individu généré au hasard'''
    
    # Un point d'intérêt de moins que la taille de la matrice des distances
    n = len(d)-1
    
    chemin = list(range(n))
    random.shuffle(chemin)
    
    return longueur_chemin(chemin,d), chemin

# Fonction principale

def créer_population(m:int, d:np.ndarray) -> list:
    
    population = []
    
    for k in range(m):
        individu = créer_individu(d)
        population.append(individu)
    
    return population

*Attention, la fonction `random.chemin(chemin)` modifie `chemin` mais ne retourne aucun résultat. Il ne faut donc pas réaffecter le résultat à `chemin` (ligne 12).*

## *III.B - Sélection*

On peut utiliser ici la méthode native `.sort()` qui permet de trier (*en la modifiant*) une liste, contrairement à la fonction `sorted()` qui renvoie une liste sans modifier la liste initiale. Pour une liste de tuples, c'est **l'ordre lexicographique** qui est utilisé, il n'est donc pas nécessaire d'extraire les longueurs des chemins car la longueur est le premier élément du couple définissant un individu. 

In [ ]:
def réduire(p:list) -> list:
    p.sort()
    p[:] = p[:len(p)//2] # On doit modifier p

*Remarque.* Lorsque l'on veut modifier un objet mutable (comme une liste) dans une fonction, on ne peut pas utiliser l'affectation :

    p = p[:len(p)//2]

car toute variable créée dans le corps d'une fonction est **locale**. Ainsi, la ligne précédente doit se comprendre :  
$$
\texttt{p}_{\texttt{loc}} = \texttt{p[:len(p)//2]}
$$
Dès lors, ce n'est pas $\texttt{p}$ qui est modifié, mais la variable locale $\texttt{p}_{\texttt{loc}}$. Pour contourner se problème, il faut utiliser l'affectation *élément par élément* :

    p[i]=...

Dans notre cas, le *slicing* $\texttt{p[:]}$ signifie *tous les éléments de $\texttt{p}$*. 

## *III.C - Mutation* 

**III.C.1)** On utilise les fonctions rappelées en annexe. La remarque de la question précédente reste valable. 

In [ ]:
def muter_chemin(c:list) -> list:   
    n = len(c)
    [i,j] = random.sample(range(n),2)

    c[i], c[j] = c[j], c[i]

In [ ]:
def muter_population(p:list, proba:float, d:np.ndarray) -> None:
    
    for i in range(len(p)):
        
        # Simulation d'une loi de Bernoulli de paramètre p
        if random.random()<= proba:
            chemin = p[i][1]
            muter_chemin(chemin)
            p[i] = longueur_chemin(chemin,d),chemin
            

*la ligne 7 simule une **loi de Bernoulli de paramètre 'proba'** car l'évènement `random.random()<=p` a pour probabilité $p$.*

## *III.D - Croisement*

**III.D.1)**

In [ ]:
def croiser(c1:list, c2:list) -> list:
    n = len(c1)    
    chemin = c1[:n//2] + c2[n//2:]
    chemin = normaliser_chemin(chemin,n)    
    return chemin 

**III.D.2)**

In [ ]:
def nouvelle_génération(p:list, d:np.array) -> None:
    
    génération=[]
     
    for i in range(len(p)):
        
        # Chemins des individus i et i+1 (modulo len(p))
        c1, c2 = p[i][1], p[(i+1)%len(p)][1]
    
        c = croiser(c1, c2)
        individu = longueur_chemin(c,d),c
        
        p.append(individu)

*Noter l'utilisation, très utile, des opérateurs modulo `%` (deuxième fonction, ligne 9) et quotient `//` (première fonction, ligne 7).*

## *III.E - Algorithme complet*

**III.E.1)**

In [ ]:
def algo_génétique(PI:np.array, m:int, proba: float, g:int) -> (float, list):
    
    d = calculer_distances(PI)
    p = créer_population(m,d)
    
    # Evolution de la population sur g génération
    for k in range(g):
        # Sélection
        réduire(p) 
        # Croisement
        nouvelle_génération(p,d)
        # Mutation
        muter_population(p, proba, d)
        # Sélection du meilleur individu
        individu = sorted(p)[1] # tri lexicographique par ordre croissant
    
    return individu

**III.2.E)** Il est possible que le chemin trouver à l'étape $n$ soit le meilleur (ou un des meilleur). Dans ce cas, la mutation peut tout à fait rendre le résultat à l'étape $n+1$ moins bon qu'à l'étape $n$ (il suffit que le bon chemin mutte en un moins bon, et que les autres ne muttent pas). Pour que cela ne se produise pas, on peut envisager un test qui retient une population mutée uniquement si celle-ci fournit un meilleur candidat, ou ne pas faire mutter le meilleur chemin.

**III.E.3)** On peut envisager d'autres critères d'arrêts :  
- pour éviter les minimums locaux, on peut s'arrêter quand la génération $n+G$ (avec $G$ fixée) n'est pas meilleure que la génération $n$, ou losrque la différenre relative ($\frac{\Delta X}{X}$) est faible,
- on peut aussi prendre en compte l'ensemble de la polpulation, i.e. considérer également le pire individu, et s'arrêter lorsque la différence relative  est plus petite qu'une quantité donnée. On s'arrête donc quand l'algorithme de croissement cesse d'être efficace (population homogène).

Ces méthodes présentes l'inconvénient de ne pas fournir à priori de borne pour le temps de calcul (*tant que...*).